In [13]:
!pip install rdflib

In [14]:

from rdflib import Graph, Literal, RDF, RDFS, Namespace, URIRef

#Namespaces
SCHEMA = Namespace("http://schema.org/")
KG = Namespace("http://kg-course.io/food-nutrition/")

g = Graph()
g.bind("schema", SCHEMA)
g.bind("ex", KG)

#Classes
classes = [
    (KG.Recipe, SCHEMA.Recipe, "A food recipe"),
    (KG.Restaurant, SCHEMA.FoodEstablishment, "An enstablishment that serves food"),
    (KG.Nutrition, SCHEMA.NutritionInformation, "Nutritional facts about a recipe"),
    (KG.Review, SCHEMA.Review, "A user review")
]

for uri, s_type, comment in classes:
    g.add((uri, RDF.type, RDFS.Class))
    g.add((uri, RDFS.subClassOf, s_type))
    g.add((uri, RDFS.comment, Literal(comment)))


with open("vocabulary.ttl", "w") as f:
    f.write(g.serialize(format="turtle"))

print("Vocabulary defined")


Vocabulary defined


In [15]:
import pandas as pd
from rdflib import Graph, Literal, RDF, Namespace, URIRef
from rdflib.namespace import XSD

# graph and namespaces
g = Graph()
SCHEMA = Namespace("http://schema.org/")
KG = Namespace("http://kg-course.io/food-nutrition/")
g.bind("schema", SCHEMA)
g.bind("ex", KG)

recipes_df = pd.read_csv("data\cleaned_recipes.csv", sep=';', engine='python', on_bad_lines='skip').head(10000)
restaurants_df = pd.read_csv("data\cleaned_restaurants.csv", sep=';', engine='python', on_bad_lines='skip').head(10000)
nutrition_df = pd.read_csv("data\cleaned_nutrition.csv", sep=';', engine='python', on_bad_lines='skip').head(10000)

print("Data loaded successfully ")

def create_kg():
    for index, row in recipes_df.iterrows():
        recipe_uri = KG[f"recipe_{row['RecipeId']}"]
        g.add((recipe_uri, RDF.type, SCHEMA.Recipe))
        g.add((recipe_uri, SCHEMA.name, Literal(row['Name'])))
        g.add((recipe_uri, SCHEMA.recipeCategory, Literal(row['RecipeCategory'])))
        g.add((recipe_uri, SCHEMA.cookTime, Literal(row['CookTime'])))

        nutri_uri = KG[f"nutrition_{row['RecipeId']}"]
        g.add((recipe_uri, SCHEMA.nutrition, nutri_uri))
        g.add((nutri_uri, RDF.type, SCHEMA.NutritionInformation))

    for index, row in restaurants_df.iterrows():
        rest_uri = KG[f"restaurant_{row['Restaurant.ID']}"]
        g.add((rest_uri, RDF.type, SCHEMA.FoodEstablishment))
        g.add((rest_uri, SCHEMA.name, Literal(row['Restaurant.Name'])))
        g.add((rest_uri, SCHEMA.address, Literal(row['Address'])))
        g.add((rest_uri, SCHEMA.servesCuisine, Literal(row['Cuisines'])))




    g.serialize(destination='KEN4256-structured-KG-Team6.ttl', format='turtle')



Data loaded successfully 
